# Structured Names
This notebook describes how to use structured names to check SBML models for moiety balance.

In [1]:
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common import simple_sbml
from SBMLLint.common.reaction import Reaction
from SBMLLint.tools import sbmllint
from SBMLLint.tools import print_reactions

import os
import pandas as pd
import tellurium as te

In [2]:
# The first step in glycolysis
model = """
Glu + A_P_P_P -> Glu_P + A_P_P; 1
"""
_ = sbmllint.lint(model)

0/1 reactions are impbalanced.
0 reactions have imbalances.



In [3]:
# The first step in glycolysis, with an error (wrongly goes to A instead of A_P_P)
model = """
Glu + A_P_P_P -> Glu_P + A; 1
"""
_ = sbmllint.lint(model)

1/1 reactions are impbalanced.
1 reactions have imbalances.

***_J0: Glu + A_P_P_P -> Glu_P + A
Excess moieties in reactants
  P: 2.00




## Sturctured Names in BioModels

Find models that already use "\_" as a name separator

In [4]:
sbmliter = simple_sbml.modelIterator(final=200)
for item in sbmliter:
    simple = simple_sbml.SimpleSBML(item.model)
    Reaction.initialize(simple)
    for reaction in Reaction.reactions:
        molecules = set(reaction.reactants).union(reaction.products)
        if any([cn.MOIETY_SEPARATOR in m.name for m in molecules]):
            path = os.path.join(cn.DATA_DIR, item.filename)
            print("\n\n***%s\n" % item.filename)
            #print(molecules)
            print_reactions.prettyPrint(path, is_include_kinetics=False)
            break
            



***BIOMD0000000198_url.xml

r1fast: NO + sGCfast -> NO_sGCfast
r2fast: NO_sGCfast -> NO_sGCfast_6coord
r3fast: NO_sGCfast_6coord -> NO_sGCfast_5coord
r1slow: NO + sGCslow -> NO_sGCslow
r2slow: NO_sGCslow -> NO_sGCslow_6coord
r3slow: NO + NO_sGCslow_6coord -> NO_sGCslow_6coord_NO_int
r4slow: NO_sGCslow_6coord_NO_int -> NO_sGCslow_5coord


***BIOMD0000000352_url.xml

reac_DIA: A + I -> D_IA
reac_DII: I + I -> D_II
reac_degrI: I -> 
reac_degrA: A -> 
reac_degrDIA: D_IA -> 
reac_degrDII: D_II -> 
reac_DIAxA: D_IA -> A
reac_DIIxI: D_II -> I
reac_degrR: R -> 
reac_prodI:  -> I
reac_prodA:  -> A
reac_prodR:  -> R


***BIOMD0000000570_url.xml

reaction_1:  -> species_1
reaction_2: 3.00 species_1 + species_2 -> species_3
reaction_3: 100.00 species_17 -> species_4
reaction_4: species_4 + species_2 -> 2.00 species_5 + species_3
reaction_5: species_5 + species_6 -> species_7 + species_9
reaction_6: species_3 + species_9 -> species_2 + species_8
reaction_7: species_8 + species_7 -> species_10 + s



***BIOMD0000000189_url.xml

Mdm2Synthesis: Mdm2_mRNA -> Mdm2_mRNA + Mdm2 + mdm2syn
Mdm2mRNASynthesis: p53 -> p53 + Mdm2_mRNA + Mdm2mRNAsyn
Mdm2mRNADegradation: Mdm2_mRNA -> Sink + Mdm2mRNAdeg
Mdm2Degradation: Mdm2 -> Sink + mdm2deg
p53Synthesis: Source -> p53 + p53syn
p53Degradation: Mdm2_p53 -> Mdm2 + p53deg
P53_Mdm2Binding: p53 + Mdm2 -> Mdm2_p53
P53_Mdm2Release: Mdm2_p53 -> p53 + Mdm2
DNAdamage:  -> damDNA + totdamDNA
DNArepair: damDNA -> Sink
ARFactivation: damDNA -> damDNA + ARF
ARF_Mdm2Binding: ARF + Mdm2 -> ARF_Mdm2
ARF_Mdm2Degradation: ARF_Mdm2 -> ARF + mdm2deg
ARFDegradation: ARF -> Sink


***BIOMD0000000140_url.xml

v1: NFkB + IkBalpha -> IkBalpha_NFkB
v2: NFkB + IkBbeta -> IkBbeta_NFkB
v3: NFkB + IkBeps -> IkBeps_NFkB
v4: NFkB + IKK_IkBalpha -> IKK_IkBalpha_NFkB
v5: IKK_IkBalpha_NFkB -> NFkB + IKK
v6: NFkB + IKK_IkBbeta -> IKK_IkBbeta_NFkB
v7: IKK_IkBbeta_NFkB -> NFkB + IKK
v8: NFkB + IKK_IkBeps -> IKK_IkBeps_NFkB
v9: IKK_IkBeps_NFkB -> NFkB + IKK
v10: IkBalpha_NFkB -> NFk

R1: mwf82ad06a_b8aa_40fa_a532_a1da44e3425f -> NGFR + mwf82ad06a_b8aa_40fa_a532_a1da44e3425f
R3: L_NGFR -> pTrkA
R4: pTrkA -> pTrkA_endo
R5: Shc_pTrkA -> Shc_pTrkA_endo
R6: pShc_pTrkA -> pShc_pTrkA_endo
R7: Grb2_SOS_pShc_pTrkA -> Grb2_SOS_pShc_pTrkA_endo
R8: FRS2_pTrkA -> FRS2_pTrkA_endo
R9: pFRS2_pTrkA -> pFRS2_pTrkA_endo
R10: Crk_C3G_pFRS2_pTrkA -> Crk_C3G_pFRS2_pTrkA_endo
R11: pTrkA_endo -> 
R12: Shc_pTrkA_endo -> Shc
R13: pShc_pTrkA_endo -> pShc
R14: Grb2_SOS_pShc_pTrkA_endo -> Grb2_SOS_pShc
R15: FRS2_pTrkA_endo -> FRS2
R16: pFRS2_pTrkA_endo -> pFRS2
R17: Crk_C3G_pFRS2_pTrkA_endo -> Crk_C3G + pFRS2
R18: pTrkA + Shc -> Shc_pTrkA
R19: pTrkA + pShc -> pShc_pTrkA
R20: pTrkA + Grb2_SOS_pShc -> Grb2_SOS_pShc_pTrkA
R21: pTrkA_endo + Shc -> Shc_pTrkA_endo
R22: pTrkA_endo + pShc -> pShc_pTrkA_endo
R23: pTrkA_endo + Grb2_SOS_pShc -> Grb2_SOS_pShc_pTrkA_endo
R24: pTrkA + FRS2 -> FRS2_pTrkA
R25: pTrkA + pFRS2 -> pFRS2_pTrkA
R26: pTrkA_endo + FRS2 -> FRS2_pTrkA_endo
R27: pTrkA_endo + pFRS2 -> pF



***BIOMD0000000293_url.xml

UbSynthesis: Source -> Ub
UbDegradation: Ub + Proteasome -> Proteasome
UbUpregulation: MisP -> MisP + 3.00 Ub + 3.00 upregUb
ProteinSynthesis: Source -> NatP
Misfolding: NatP + ROS -> MisP + ROS
Refolding: MisP -> NatP
MisPE3Binding: MisP + E3 -> E3_MisP
MisPE3Release: E3_MisP -> MisP + E3
E1UbBinding: E1 + Ub + ATP -> E1_Ub + AMP
E2UbBinding: E2 + E1_Ub -> E2_Ub + E1
Monoubiquitination: E2_Ub + E3_MisP -> E3_MisP_Ub + E2
Polyubiquitination1: E3_MisP_Ub + E2_Ub -> E3_MisP_Ub2 + E2
Polyubiquitination2: E3_MisP_Ub2 + E2_Ub -> E3_MisP_Ub3 + E2
Polyubiquitination3: E3_MisP_Ub3 + E2_Ub -> E3_MisP_Ub4 + E2
Polyubiquitination4: E3_MisP_Ub4 + E2_Ub -> E3_MisP_Ub5 + E2
Polyubiquitination5: E3_MisP_Ub5 + E2_Ub -> E3_MisP_Ub6 + E2
Polyubiquitination6: E3_MisP_Ub6 + E2_Ub -> E3_MisP_Ub7 + E2
Polyubiquitination7: E3_MisP_Ub7 + E2_Ub -> E3_MisP_Ub8 + E2
MisPDUBbinding1: E3_MisP_Ub + DUB -> E3_MisP_Ub_DUB
MisPDUBbinding2: E3_MisP_Ub2 + DUB -> E3_MisP_Ub2_DUB
MisPDUBbindi

binding_CRP_cAMP: CRP + cAMP -> CRP_cAMP
binding_CRP_cAMP_CRPsite_cyaA: CRP_cAMP + CRPsite_cyaA -> CRP_cAMP_CRPsite_cyaA
binding_CRP_cAMP_CRPsiteI_crp: CRP_cAMP + CRPsiteI_crp -> CRP_cAMP_CRPsiteI_crp
binding_CRP_cAMP_CRPsiteII_crp: CRP_cAMP + CRPsiteII_crp -> CRP_cAMP_CRPsiteII_crp
binding_CRP_cAMP_CRPsite_ptsGp1: CRP_cAMP + CRPsite_ptsGp1 -> CRP_cAMP_CRPsite_ptsGp1
binding_CRP_cAMP_CRPsite_ptsGp2: CRP_cAMP + CRPsite_ptsGp2 -> CRP_cAMP_CRPsite_ptsGp2
binding_CRP_cAMP_CRPsite_ptsHp0: CRP_cAMP + CRPsite_ptsHp0 -> CRP_cAMP_CRPsite_ptsHp0
binding_CRP_cAMP_CRPsite_ptsHp1: CRP_cAMP + CRPsite_ptsHp1 -> CRP_cAMP_CRPsite_ptsHp1
binding_CRP_cAMP_CRPsite_ptsIp0: CRP_cAMP + CRPsite_ptsIp0 -> CRP_cAMP_CRPsite_ptsIp0
binding_CRP_cAMP_CRPsite_ptsIp1: CRP_cAMP + CRPsite_ptsIp1 -> CRP_cAMP_CRPsite_ptsIp1
binding_CRP_cAMP_CRPsite_mlcp1: CRP_cAMP + CRPsite_mlcp1 -> CRP_cAMP_CRPsite_mlcp1
binding_CRP_cAMP_CRPsite_mlcp2: CRP_cAMP + CRPsite_mlcp2 -> CRP_cAMP_CRPsite_mlcp2
binding_CRP_cAMP_CRPsite_genome: C

...



***BIOMD0000000167_url.xml

PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
statDephosphorylation: Pstat_nuc -> stat_nuc
statPhosphorylation: stat_sol -> Pstat_sol + species_test
PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc
PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc
stat_export: stat_sol -> stat_nuc
stat_import: stat_sol -> stat_nuc


***BIOMD0000000122_url.xml

R1: NFAT_Pi_Nuc + Act_C_Nuc -> Act_C_Nuc + NFAT_Nuc
R2: Act_C_Nuc + NFAT_Nuc -> NFAT_Act_C_Nuc
R3: NFAT_Nuc -> NFAT_Cyt
R4: Act_C_Nuc -> Act_C_Cyt
R5: NFAT_Act_C_Nuc -> NFAT_Pi_Act_C_Nuc
R6: NFAT_Pi_Act_C_Nuc -> Act_C_Nuc + NFAT_Pi_Nuc
R7: NFAT_Act_C_Nuc -> NFAT_Act_C_Cyt
R8: NFAT_Act_C_Cyt -> NFAT_Pi_Act_C_Cyt
R9: NFAT_Pi_Act_C_Cyt -> Act_C_Cyt + NFAT_Pi_Cyt
R10: NFAT_Pi_Cyt -> NFAT_Pi_Nuc
R11: NFAT_Act_C_Cyt -> Act_C_Cyt + NFAT_Cyt
R17: NFAT_Pi_Act_C_Cyt -> NFAT_Pi_Act_C_Nuc
R12: NFAT_Pi_Cyt + Act_C_Cyt -> Act_C_Cyt + NFAT_Cyt
R13: 3.00 Ca_Cyt + Inact_C_Cyt -> Act_C_Cyt
R14: 3.00 Ca_Nuc + Inact_C_Nuc



***BIOMD0000000272_url.xml

reaction_1:  -> EpoR
reaction_2: EpoR -> 
reaction_3: SAv + EpoR -> SAv_EpoR
reaction_4: SAv_EpoR -> SAv + EpoR
reaction_5: SAv_EpoR -> SAv_EpoRi
reaction_6: SAv_EpoRi -> SAv
reaction_7: SAv_EpoRi -> dSAvi
reaction_8: SAv_EpoRi -> dSAve


***BIOMD0000000245_url.xml

r1: s_glu -> 0.98 s_pyr + 0.18 Red
r2: s_pyr -> 1.50 CO2 + 0.91 Red
r3: s_pyr -> 0.50 s_acetald + 0.33 CO2
r4: s_acetald -> 1.36 s_acetate + 0.36 Red
r5: s_acetate -> 1.45 CO2 + 1.07 Red
r6: s_acetald + 0.36 Red -> 1.04 s_EtOH
r7: s_glu -> 0.73 x + 0.13 CO2 + 0.06 Red
r8: s_acetate -> 0.62 x + 0.33 CO2 + 0.21 Red
r9: a -> AcDH
r10: a -> 
r11: AcDH -> 
s_glu_in: S_f -> s_glu
s_glu_out: s_glu -> 
s_pyr_out: s_pyr -> 
s_acetate_out: s_acetate -> 
s_acetald_out: s_acetald -> 
s_EtOH_out: s_EtOH -> 
a_out: a -> 
x_out: x -> 
AcDH_out: AcDH -> 


***BIOMD0000000340_url.xml

r1_: VIII -> VIIIa
r2_: VIIIa -> 
r3_: IX -> IXa
r4_: XI -> XIa
r5_: XI -> XIa
r6_: VII -> VIIa
r7_: X -> Xa
r8_: X -> Xa
r9_: X



***BIOMD0000000162_url.xml

CD28k_highDbinding: Ca_D_Cytosol + D28k_high_D_Cytosol -> D28kB_high_D_Cytosol
PA_Dbinding: PA_D_Cytosol + Ca_D_Cytosol -> PABCa_D_Cytosol
CG_deg: CG_D_Cytosol -> 
IP3deg: IP3_D_Cytosol -> 
D28kB_deg: D28kB_D_Cytosol -> 
D28k_deg: D28k_D_Cytosol -> 
PA_MgD: Mg_D_Cytosol + PA_D_Cytosol -> PABMg_D_Cytosol
PA_Ca: PA_Cytosol + Ca_Cytosol -> PABCa_Cytosol
PABMg_deg: PABMg_D_Cytosol -> 
D28k_high_deg: D28k_high_D_Cytosol -> 
Ca_d: Ca_Cytosol -> 
D28kB_high_deg: D28kB_high_D_Cytosol -> 
CG_d: CG_Cytosol -> 
D28kB_d: D28kB_Cytosol -> 
reaction1: h_D_ERM -> 
CGbinding: Ca_Cytosol + CG_Cytosol -> CGB_Cytosol
CaD28k_med: D28k_Cytosol + Ca_Cytosol -> D28kB_Cytosol
PABCa_deg: PABCa_D_Cytosol -> 
IP3_d: IP3_Cytosol -> 
D28k_high_d: D28k_high_Cytosol -> 
reaction0: h_ERM -> 
PABMg_d: PABMg_Cytosol -> 
CD28k_high: Ca_Cytosol + D28k_high_Cytosol -> D28kB_high_Cytosol
D28kBDbinding: D28k_D_Cytosol + Ca_D_Cytosol -> D28kB_D_Cytosol
PABCa_d: PABCa_Cytosol -> 
PA_d: PA_Cytosol



***BIOMD0000000410_url.xml

_29: species_27 -> _5 + 2.00 species_17
_31: _9 -> species_19
_33: _9 + 2.00 _11 -> _13
_35: 3.00 _15 + species_13 -> 3.00 _19
_37: _5 + 2.00 _19 -> _9
_39: _19 -> _21
_41: _21 + _15 -> 
_43: _15 + _25 -> _27
_77: 2.00 _75 -> _79
_86: _79 + 2.00 _84 -> _96
_103: _101 + _99 -> _105
_125: _99 -> _129
_132: _105 -> _129 + _101
_139: _129 -> _19
_149: _147 + 2.00 _129 -> _153
_156: _147 -> _5
_164: _153 -> _9
_172: _25 -> _174
_178: _27 -> _181
_192: _181 + _96 -> 
_194: _174 + _96 -> _198
reaction_1:  -> _75
reaction_2:  -> _101
reaction_3:  -> _174
reaction_4:  -> species_1
reaction_5:  -> species_2
reaction_6:  -> _84
reaction_7: _5 + _11 -> species_16
reaction_8: _99 -> _21
reaction_9: _96 -> 
reaction_10: _84 -> 
reaction_11: _75 -> 
reaction_12: _79 -> 2.00 _75
reaction_13:  -> _99
reaction_14:  -> _147
reaction_15: _147 -> 
reaction_16: _101 -> 
reaction_17: _129 -> _99
reaction_18: species_2 -> 
reaction_19: _15 -> species_1
reaction_20: _99 -> 
reacti

TPI_g: DHAP_g -> GA3P_g
PYK_c: PEP_c + ADP_c -> Pyr_c + ATP_c
PFK_g: ATP_g + Fru6P_g -> Fru16BP_g + ADP_g
G6PDH_g: Glc6P_g + NADP_g -> _6PGL_g + NADPH_g
PGAM_c: _3PGA_c -> _2PGA_c
PyrT_c: Pyr_c -> Pyr_e
G6PDH_c: Glc6P_c + NADP_c -> NADPH_c + _6PGL_c
ENO_c: _2PGA_c -> PEP_c
HXK_g: ATP_g + Glc_g -> Glc6P_g + ADP_g
_3PGAT_g: _3PGA_g -> _3PGA_c
NADPHu_c: NADPH_c -> NADP_c
HXK_c: Glc_c + ATP_c -> Glc6P_c + ADP_c
NADPHu_g: NADPH_g -> NADP_g
AK_c: 2.00 ADP_c -> AMP_c + ATP_c
G6PP_c: Glc6P_c -> Glc_c
_6PGDH_g: _6PG_g + NADP_g -> Rul5P_g + CO2_g + NADPH_g
PGI_g: Glc6P_g -> Fru6P_g
AK_g: 2.00 ADP_g -> AMP_g + ATP_g
TOX_c: TSH2_c -> TS2_c
GDA_g: Gly3P_g + DHAP_c -> Gly3P_c + DHAP_g
ATPT_g: ADP_g + ATP_c -> ATP_g + ADP_c
PGL_c: _6PGL_c -> _6PG_c
_6PGDH_c: NADP_c + _6PG_c -> CO2_c + NADPH_c + Rul5P_c
PPI_c: Rul5P_c -> Rib5P_c
PPI_g: Rul5P_g -> Rib5P_g
GlcT_g: Glc_c -> Glc_g
GlcT_c: Glc_e -> Glc_c
PGL_g: _6PGL_g -> _6PG_g
TR_c: TS2_c + NADPH_c -> NADP_c + TSH2_c
PGK_g: _13BPGA_g + ADP_g -> _3PGA_g +

In [5]:
# Analysis of BIOMD0000000611_url.xml
filename = "BIOMD0000000611_url.xml"
filename = "BIOMD0000000198_url.xml"
filename = "BIOMD0000000140_url.xml"
filename = "BIOMD0000000611_url.xml"
filename = "BIOMD0000000139_url.xml"
filename = "BIOMD0000000362_url.xml" # 16/33 don't comply
filename = "BIOMD0000000571_url.xml" 

path = os.path.join(cn.DATA_DIR, filename)
sbmllint.lint(path)

65/92 reactions are impbalanced.
65 reactions have imbalances.

***transcription_CRP_cAMP_CRPsite_cyaA_cyaA:  -> mRNA_cyaA
Excess moieties in products
  cyaA: 1.00
  mRNA: 1.00


***transcription_cyaA_basal:  -> mRNA_cyaA
Excess moieties in products
  cyaA: 1.00
  mRNA: 1.00


***transcription_CRP_cAMP_CRPsiteI_crp_CRP_cAMP_CRPsiteII_crp_crp:  -> mRNA_crp
Excess moieties in products
  crp: 1.00
  mRNA: 1.00


***transcription_crp_basal:  -> mRNA_crp
Excess moieties in products
  crp: 1.00
  mRNA: 1.00


***transcription_CRP_cAMP_CRPsite_ptsGp1_Mlc_Mlcsite_ptsGp1_ptsGp1:  -> mRNA_ptsG
Excess moieties in products
  mRNA: 1.00
  ptsG: 1.00


***transcription_CRP_cAMP_CRPsite_ptsGp2_Mlc_Mlcsite_ptsGp2_ptsGp2:  -> mRNA_ptsG
Excess moieties in products
  mRNA: 1.00
  ptsG: 1.00


***transcription_CRP_cAMP_CRPsite_ptsHp0_Mlc_Mlcsite_ptsHp0_ptsHp0:  -> mRNA_ptsH
Excess moieties in products
  mRNA: 1.00
  ptsH: 1.00


***transcription_CRP_cAMP_CRPsite_ptsHp1_ptsHp1:  -> mRNA_ptsH
Excess moietie

(92, 65)

**To do:**
1. Table of counts of total reactions, number imbalance (less boundary reactions). Do as bar plot.

In [7]:
# Construct dataframe of statistics on names and imbalances
FILENAME = "filename"
HAS_DASH = "has_dash"
NUM_REACTIONS = "num_reactions"
NUM_BOUNDARY_REACTIONS = "num_boundary_reactions"
NUM_BAD = "num_imbalance_reactions"
dfs = []
sbmliter = simple_sbml.modelIterator(final=700)
for item in sbmliter:
    simple = simple_sbml.SimpleSBML(item.model)
    row = {FILENAME: [item.filename], 
           HAS_DASH: [False], 
           NUM_BOUNDARY_REACTIONS: [0],
           NUM_REACTIONS: [0],
           NUM_BAD: [None],
           NUM_BAD: [0],
           }
    Reaction.initialize(simple)
    for reaction in Reaction.reactions:
        if (len(reaction.reactants) == 0) or (len(reaction.products) == 0):
            row[NUM_BOUNDARY_REACTIONS] = [row[NUM_BOUNDARY_REACTIONS][0] + 1]
        molecules = set(reaction.reactants).union(reaction.products)
        if any([cn.MOIETY_SEPARATOR in m.name for m in molecules]):
            row[HAS_DASH] = [True]
    num_reactions, num_bad = sbmllint.lint(item.model, is_report=False)
    row[NUM_REACTIONS] = [num_reactions]
    row[NUM_BAD] = [num_bad]
    dfs.append(pd.DataFrame(row))
df_count = pd.concat(dfs)

KeyboardInterrupt: 

In [ ]:
df_count
